<a href="https://colab.research.google.com/github/ADIthaker/CNN-Techniques/blob/master/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!unzip -q flowers17.zip -d ~/.keras/datasets

In [5]:
import os
import pathlib
from glob import glob

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras import Sequential
from tensorflow_hub import KerasLayer

In [2]:
SEED= 999

In [3]:
def build_network(base_model, classes):
  x = Flatten()(base_model.output)
  x = Dense(units=256)(x)
  x = ReLU()(x)
  x = BatchNormalization(axis=-1)(x)
  x = Dropout(rate=0.5)(x)

  x = Dense(units=classes)(x)
  output = Softmax()(x)

  return output

In [6]:
def build_network_tfhub(base_model, classes):
  return Sequential([
                     base_model,
                     Dense(classes),
                     Softmax()
  ])

In [4]:
def load_images_and_labels(image_paths, target_size=(256,256)):
  images= []
  labels = []
  for image_path in image_paths:
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    label = image_path.split(os.path.sep)[-2]
    images.append(image)
    labels.append(label)
  return np.array(images), np.array(labels)

In [8]:
augmenter = ImageDataGenerator(rotation_range=30,
                               horizontal_flip=True,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.2,
                               zoom_range=0.2,
                               fill_mode='nearest')

In [10]:
dataset_path = (pathlib.Path.home()/ '.keras' / 'datasets'/ 'flowers17')
files_pattern = (dataset_path / 'images'/ '*'/ '*.jpg')
image_paths = [*glob(str(files_pattern))]
CLASSES  = {p.split(os.path.sep)[-2] for p in image_paths}

In [11]:
X, y = load_images_and_labels(image_paths)
X = X.astype('float') / 255.0
y = LabelBinarizer().fit_transform(y)

(X_train, X_test,
 y_train, y_test) = train_test_split(X, y,
                                     test_size=0.2,
                                     random_state=SEED)

In [12]:
base_model = VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(256,256,3)))

58900480/58889256 [==============================] - 1s 0us/step


In [14]:
for layer in base_model.layers:
  layer.trainable = False

In [15]:
model = build_network(base_model, len(CLASSES))
model = Model(base_model.input, model)

In [16]:
BATCH_SIZE = 64
train_generator = augmenter.flow(X_train, y_train, BATCH_SIZE)

In [18]:
WARMING_EPOCHS = 20
model.compile(loss='categorical_crossentropy',
optimizer=RMSprop(learning_rate=1e-3), metrics=['accuracy'])

In [19]:
model.fit(train_generator,
steps_per_epoch=len(X_train) // BATCH_SIZE,
validation_data=(X_test, y_test),
epochs=WARMING_EPOCHS)
result = model.evaluate(X_test, y_test)
print(f'Test accuracy: {result[1]}')

Epoch 1/20
17/17 [==============================] - 81s 2s/step - loss: 1.5377 - accuracy: 0.5561 - val_loss: 4.1264 - val_accuracy: 0.3824
Epoch 2/20
17/17 [==============================] - 21s 1s/step - loss: 0.6926 - accuracy: 0.7960 - val_loss: 2.2110 - val_accuracy: 0.5257
Epoch 3/20
17/17 [==============================] - 21s 1s/step - loss: 0.4725 - accuracy: 0.8658 - val_loss: 1.1904 - val_accuracy: 0.6765
Epoch 4/20
17/17 [==============================] - 21s 1s/step - loss: 0.3886 - accuracy: 0.8943 - val_loss: 0.8634 - val_accuracy: 0.7610
Epoch 5/20
17/17 [==============================] - 21s 1s/step - loss: 0.3581 - accuracy: 0.8943 - val_loss: 0.8365 - val_accuracy: 0.7757
Epoch 6/20
17/17 [==============================] - 21s 1s/step - loss: 0.2776 - accuracy: 0.9292 - val_loss: 0.9874 - val_accuracy: 0.7463
Epoch 7/20
17/17 [==============================] - 21s 1s/step - loss: 0.2720 - accuracy: 0.9366 - val_loss: 0.5887 - val_accuracy: 0.8199
Epoch 8/20
17/17 [==

In [20]:
for layer in base_model.layers[15:]:
  layer.trainable = True
EPOCHS = 50
model.compile(loss='categorical_crossentropy',
optimizer=SGD(lr=1e-3),
metrics=['accuracy'])
model.fit(train_generator,
steps_per_epoch=len(X_train) // BATCH_SIZE,
validation_data=(X_test, y_test),
epochs=EPOCHS)
result = model.evaluate(X_test, y_test)
print(f'Test accuracy: {result[1]}')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
17/17 [==============================] - 24s 1s/step - loss: 0.0998 - accuracy: 0.9761 - val_loss: 0.4688 - val_accuracy: 0.8493
Epoch 2/50
17/17 [==============================] - 21s 1s/step - loss: 0.0859 - accuracy: 0.9881 - val_loss: 0.4070 - val_accuracy: 0.8824
Epoch 3/50
17/17 [==============================] - 21s 1s/step - loss: 0.0952 - accuracy: 0.9798 - val_loss: 0.3564 - val_accuracy: 0.9118
Epoch 4/50
17/17 [==============================] - 21s 1s/step - loss: 0.0739 - accuracy: 0.9871 - val_loss: 0.3266 - val_accuracy: 0.9228
Epoch 5/50
17/17 [==============================] - 21s 1s/step - loss: 0.0755 - accuracy: 0.9853 - val_loss: 0.3084 - val_accuracy: 0.9191
Epoch 6/50
17/17 [==============================] - 21s 1s/step - loss: 0.0806 - accuracy: 0.9862 - val_loss: 0.2876 - val_accuracy: 0.9301
Epoch 7/50
17/17 [==============================] - 21s 1s/step - loss: 0.0761 - accuracy: 0.9862 - val_loss: 0.2748 - val_accuracy: 0.9375
Epoch 8/50
17/17 [==

In [21]:
model_url = ('https://tfhub.dev/google/imagenet/'
             'resnet_v1_152/feature_vector/4')
base_model = KerasLayer(model_url, input_shape=(256, 256, 3))
base_model.trainable = False

model = build_network_tfhub(base_model, len(CLASSES))
BATCH_SIZE = 32
augmenter = ImageDataGenerator(horizontal_flip=True,
                               rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.2,
                               zoom_range=0.2,
                               fill_mode='nearest')
train_generator = augmenter.flow(X_train, y_train, BATCH_SIZE)
EPOCHS = 20
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=1e-3),
              metrics=['accuracy'])
history = model.fit(train_generator,
                    steps_per_epoch=len(X_train) // BATCH_SIZE,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS)
result = model.evaluate(X_test, y_test)
print(f'Test accuracy: {result[1]}')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
34/34 [==============================] - 42s 842ms/step - loss: 1.6109 - accuracy: 0.5312 - val_loss: 0.7363 - val_accuracy: 0.8272
Epoch 2/20
34/34 [==============================] - 25s 744ms/step - loss: 0.5619 - accuracy: 0.8621 - val_loss: 0.4250 - val_accuracy: 0.8934
Epoch 3/20
34/34 [==============================] - 25s 745ms/step - loss: 0.3466 - accuracy: 0.9127 - val_loss: 0.3181 - val_accuracy: 0.9154
Epoch 4/20
34/34 [==============================] - 26s 751ms/step - loss: 0.2397 - accuracy: 0.9449 - val_loss: 0.2690 - val_accuracy: 0.9118
Epoch 5/20
34/34 [==============================] - 25s 743ms/step - loss: 0.1814 - accuracy: 0.9596 - val_loss: 0.2520 - val_accuracy: 0.9118
Epoch 6/20
34/34 [==============================] - 25s 744ms/step - loss: 0.1525 - accuracy: 0.9577 - val_loss: 0.1933 - val_accuracy: 0.9449
Epoch 7/20
34/34 [==============================] - 26s 750ms/step - loss: 0.1251 - accuracy: 0.9642 - val_loss: 0.1753 - val_accuracy: 0.9449